In [97]:
!pip install coremltools
!pip install matplotlib

     |████████████████████████████████| 12.4 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 60 kB 6.3 MB/s eta 0:00:011
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)


In [98]:
import PIL
from PIL import Image
%matplotlib inline

In [2]:
import coremltools

In [3]:
!echo $VIRTUAL_ENV

/Users/arthurtonelli/.virtualenvs/coreml_fst


In [4]:
!ls $(which python)

/Users/arthurtonelli/.virtualenvs/coreml_fst/bin/python


In [5]:
rose = Image.open('/Users/arthurtonelli/Downloads/rose.jpg')

In [6]:
rose_sm = rose.resize((175,380))

In [111]:
rose_bg = rose.resize((350,760))

In [7]:
model = coremltools.models.MLModel('/Users/arthurtonelli/Downloads/SketchResCompactsmRGB.mlmodel')

In [8]:
res = model.predict({'input_5': rose_sm})

RuntimeError: {
    NSLocalizedDescription = "Failed to convert output transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2 to image";
    NSUnderlyingError = "Error Domain=com.apple.CoreML Code=0 \"Invalid array shape (\n    380,\n    176,\n    3\n) for converting to gray image\" UserInfo={NSLocalizedDescription=Invalid array shape (\n    380,\n    176,\n    3\n) for converting to gray image}";
}

In [56]:
# builder = None
model_sm = coremltools.models.MLModel('/Users/arthurtonelli/Downloads/SketchResCompactsm.mlmodel')

In [57]:
res = model_sm.predict({'input_5': rose_sm})

In [58]:
res['transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2'].shape

(1, 380, 176, 3)

In [18]:
model_id = coremltools.models.MLModel('/Users/arthurtonelli/Downloads/SketchResCompactId.mlmodel')

In [19]:
res = model_id.predict({'input_5': rose_sm})

In [20]:
res['Identity'].shape

(1, 380, 176, 3)

In [59]:
spec_sm = model_sm.get_spec()

In [60]:
spec_sm.description

input {
  name: "input_5"
  type {
    imageType {
      width: 175
      height: 380
      colorSpace: RGB
    }
  }
}
output {
  name: "transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.3"
  }
}

In [61]:
builder=coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec_sm)

In [52]:
# builder.add_reshape?

In [62]:
builder.add_transpose(name="reshape_arthur", 
                    axes=(0,3,1,2), 
#                     mode=1, 
                    input_name='transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2',
                    output_name='reshape_arthur')

name: "reshape_arthur"
input: "transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2"
output: "reshape_arthur"
transpose {
  axes: 0
  axes: 3
  axes: 1
  axes: 2
}

In [76]:
builder.inspect_layers(3, verbose=True)

[Id: 41], Name: reshape_arthur (Type: transpose)
          Updatable: False
          Input blobs: ['transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2']
          Output blobs: ['reshape_arthur']
          Parameters: 
              axes = [0, 3, 1, 2]
[Id: 40], Name: transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2 (Type: transpose)
          Updatable: False
          Input blobs: ['preIdentity_transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2']
          Output blobs: ['transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2']
          Parameters: 
              axes = [0, 2, 3, 1]
[Id: 39], Name: preIdentity_transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2 (Type: multiplyBroadcastable)
          Updatable: False
          Input blobs:

In [80]:
builder.spec.description.output

[name: "transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2"
type {
  multiArrayType {
    dataType: FLOAT32
  }
}
]

In [82]:
builder.set_output(output_names=['reshape_arthur'], 
                   output_dims=[(1,3,380, 176)])

In [83]:
coremltools.utils.save_spec(builder.spec, "/Users/arthurtonelli/Downloads/SketchResCompactReshape2.mlmodel")

In [84]:
model_reshape = coremltools.models.MLModel('/Users/arthurtonelli/Downloads/SketchResCompactReshape2.mlmodel')

In [85]:
res_reshape = model_reshape.predict({'input_5': rose_sm})

In [87]:
res_reshape['reshape_arthur'].shape

(1, 3, 380, 176)

In [78]:
builder.layers

['transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_add_2/PartitionedCall/add_2/y',
 'transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2/y',
 'transformnet/StatefulPartitionedCall/StatefulPartitionedCall/conv_layer_8/StatefulPartitionedCall/conv_1/Conv2D',
 'transformnet/StatefulPartitionedCall/StatefulPartitionedCall/conv_layer_8/StatefulPartitionedCall/batch_normalization_24/FusedBatchNormV3/Add_batch_norm',
 'transformnet/StatefulPartitionedCall/StatefulPartitionedCall/conv_layer_8/StatefulPartitionedCall/activation_12/Relu',
 'transformnet/StatefulPartitionedCall/StatefulPartitionedCall/conv_layer_9/StatefulPartitionedCall/conv_2/Conv2D',
 'transformnet/StatefulPartitionedCall/StatefulPartitionedCall/conv_layer_9/StatefulPartitionedCall/batch_normalization_25/FusedBatchNormV3/Add_batch_norm',
 'transformnet/StatefulPartitionedCall/StatefulPartitionedCall/conv_layer_9/StatefulPartitionedCall/activation_13/Relu'

In [79]:
res_reshape['transformnet/StatefulPartitionedCall/StatefulPartitionedCall/tf_op_layer_mul_2/PartitionedCall/mul_2'].shape

(1, 380, 176, 3)

In [89]:
output = builder.spec.description.output[0]

import coremltools.proto.FeatureTypes_pb2 as ft
output.type.imageType.colorSpace = ft.ImageFeatureType.RGB
output.type.imageType.height = 380
output.type.imageType.width = 176

coremltools.utils.save_spec(builder.spec, "/tmp/SketchResCompRSRGB.mlmodel")

In [91]:
model_reshape_rgb = coremltools.models.MLModel('/tmp/SketchResCompRSRGB.mlmodel')

In [92]:
res_reshape_rgb = model_reshape_rgb.predict({'input_5': rose_sm})

In [93]:
res_reshape_rgb

{'reshape_arthur': <PIL.Image.Image image mode=RGBA size=176x380 at 0x15B998358>}

In [94]:
img = res_reshape_rgb['reshape_arthur']

In [99]:
img.show()

In [106]:
d = img.getdata()

In [96]:
!cp /tmp/SketchResCompRSRGB.mlmodel /Users/arthurtonelli/Downloads/SketchResCompRSRGB.mlmodel

In [113]:
model_img_at = coremltools.models.MLModel('/Users/arthurtonelli/Downloads/SketchResImgAT.mlmodel')

In [114]:
res_at = model_img_at.predict({'input_5': rose_bg})
res_at['reshape_arthur'].show()

In [115]:
model_img = coremltools.models.MLModel('/Users/arthurtonelli/Downloads/SketchResImg.mlmodel')

In [116]:
# res_img = model_img.predict({'input': rose_bg})

Exception: Unable to load CoreML.framework. Cannot make predictions.